# Example step by step DLC

First create the project

In [ ]:
import os

# os.system("module load cuDNN/8.1.1.33-CUDA-11.2.1")
os.system("module list")

In [ ]:
os.environ["LD_LIBRARY_PATH"] = "/camp/home/blota/.conda/envs/deeplabcut_conda12/lib/"
print(os.environ["LD_LIBRARY_PATH"])

In [ ]:
import tensorflow

print(tensorflow.__version__)

In [ ]:
from tensorflow.python.client import device_lib


def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]


print(get_available_devices())

In [ ]:
import deeplabcut
from pathlib import Path

config = Path("/camp/lab/znamenskiyp/home/shared/projects/DLC_models/")
config /= "WehrCam_detect_eye-ZnamLab-2023-09-05/config.yaml"

print(config)

In [ ]:
import flexiznam as flm

# project = "hey2_3d-vision_foodres_20220101"
project = "blota_onix_pilote"
flm_sess = flm.get_flexilims_session(project_id=project)

In [ ]:
# get a list of videos to add
session_dict = {
    "BRAC7448.2d": ["S20230421", "S20230412"],
    "BRYA142.5d": ["S20230825", "S20230829", "S20230830", "S20230831"],
    # "PZAH6.4b": ["S20220419", "S20220421"],
    # "PZAG3.4f": ["S20220421", "S20220517"],
}
from flexiznam.schema import Dataset
import shutil

ONIX = True
tmp_folder = Path("/nemo/lab/znamenskiyp/home/shared/projects/DLC_models/tmp")
tmp_folder.mkdir(exist_ok=True)
vids = []

for mouse, sessions in session_dict.items():
    for sess in sessions:
        fl_sess = flm.get_entity(
            name=f"{mouse}_{sess}", flexilims_session=flm_sess, datatype="session"
        )
        if fl_sess is None:
            print(f"Session {mouse}_{sess} not found")
        if ONIX:
            ds_dict = flm.get_datasets_recursively(
                origin_id=fl_sess.id, flexilims_session=flm_sess, dataset_type="camera"
            )
            ds = []
            for k, v in ds_dict.items():
                ds.extend(v)
            eye_ds = [d for d in ds if "eye_camera" in d.dataset_name]
            eye_ds = [d for d in ds if "deinterleaved" in d.dataset_name]
        else:
            if project == "hey2_3d-vision_foodres_20220101":
                recs = [r for n, r in recs.iterrows() if "Spheres" in n]
            else:
                recs = [r for n, r in recs.iterrows() if "freely" in n]
            rec = recs[0]
            ds = flm.get_children(
                parent_id=rec["id"],
                flexilims_session=flm_sess,
                children_datatype="dataset",
            )
            eye_ds = [r for n, r in ds.iterrows() if "eye_camera" in n]
            eye_ds = [r for r in eye_ds if r.dataset_type == "camera"]
            eye_ds = [
                Dataset.from_data_series(ds, flexilims_session=flm_sess)
                for ds in eye_ds
            ]
        for ds in eye_ds:

            vid = ds.path_full / ds.extra_attributes["video_file"]
            target = tmp_folder / f"{'_'.join(ds.genealogy[:-1])}_{vid.name}"
            if not target.exists():
                print(f"Copying {vid} to {target}")
                shutil.copy(vid, target)
            vids.append(target)

In [ ]:
vids

In [ ]:
vids =  ['/nemo/lab/znamenskiyp/home/shared/projects/DLC_models/tmp/BRAC7448.2d_S20230412_R162624_freelymoving_eye_camera_2023-04-12T16_26_24_deinterleaved.mp4',
         '/nemo/lab/znamenskiyp/home/shared/projects/DLC_models/tmp/BRYA142.5d_S20230831_R144404_headmounted_eye_camera_deinterleaved.mp4']

In [ ]:
# add these videos (do it only once)
if True:
    deeplabcut.add_new_videos(
        config, videos=[str(v) for v in vids], extract_frames=False, copy_videos=True
    )

In [ ]:
vids

In [ ]:
# this will reextract for all videos
if False:
    deeplabcut.extract_frames(
        config=config,
        userfeedback=True,
        #videos_list=vids,
        algo="kmeans",
    )

Edit config file manually to add the skeleton and body parts you want. Then head to the
VM to label manually. Copy the labelled data back to camp and edit config.yaml to change
paths if needed.

Then check labels

In [ ]:
deeplabcut??

In [ ]:
deeplabcut.check_labels(config, visualizeindividuals=True)

In [ ]:
_ = deeplabcut.create_training_dataset(
    config, augmenter_type="imgaug", userfeedback=False
)

Now train the network (unsing the sbatch script)

In [ ]:
o = deeplabcut.evaluate_network(config, Shuffles=[1], plotting=True)

In [ ]:
scm = deeplabcut.extract_save_all_maps(config, shuffle=1, Indices=[0, 5])